In [1]:
import pandas as pd
import re
import spacy
from collections import Counter
from bs4 import BeautifulSoup
from boilerpipe.extract import Extractor
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from nltk.corpus import stopwords
stop = set(stopwords.words())
ANNOTATION_RESULTS = "../1-Data/3-annotation/output.csv"
df = pd.read_csv(ANNOTATION_RESULTS)
nlp = spacy.load('en_core_web_sm')

Clean text from html tags

In [2]:
df = df.dropna(subset =["content"])


In [3]:
def cleanMe(html):
    soup = BeautifulSoup(html) # create a new bs4 object from the html data loaded
    for script in soup(["script", "style"]): # remove all javascript and stylesheet code
        script.extract()
    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text
df['text'] = df.apply(lambda row: "{} {}".format(row['title'], cleanMe(row['content'])), axis=1)

/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


## Company Confidence Baseline:

In [4]:
COMPANY_NAMES_STOP_WORDS = r"PLC|Corp|Corporation|—$"
df['company_confidence'].value_counts(normalize=True)

1.0    0.409548
2.0    0.339196
0.0    0.251256
Name: company_confidence, dtype: float64

The baseline is built on frequency of mentions of the company, comparing to other organizations:  
2 - Very Confident that the text is about this company. It's mention fraction => 10% and at least 2 mentions.  
1 - Moderate. It is mentioned but the fraction < 10%.  
0 - Not related to the company. The company is not mentioned in text.  
TODO: take in account how many organizations mentioned, how many words in the text etc.

In [5]:
company_confidence = []
company_fraction = []

for num, row in df.iterrows():
    company = re.sub(COMPANY_NAMES_STOP_WORDS, '', row['company'], flags=re.IGNORECASE).lower()
    #Company is often mentioned by part of it's name. e.g. "Royal Dutch Shell" -> "Shell"
    company_names = set([company] + company.split())
    text = row['text']
    doc = nlp(text)
    orgs = [re.sub(COMPANY_NAMES_STOP_WORDS, '', ent.text, flags=re.IGNORECASE).strip().lower() for ent in doc.ents if ent.label_ == "ORG"]
    orgs_counter = Counter(orgs)
    occurences_company = sum([orgs_counter.get(name, 0) for name in company_names])
    occurences_total = sum(orgs_counter.values())
    if occurences_total == 0:
        fraction = 0
    else:
        fraction = occurences_company/occurences_total
    company_fraction.append(fraction)
    if occurences_company == 0:
        company_confidence.append(0)
    elif fraction > 0.1 and occurences_company > 1:
        company_confidence.append(2)
    else:
        company_confidence.append(1)

df['company_confidence_estimation'] = company_confidence
df['company_fraction'] = company_fraction

In [6]:
print(classification_report(df['company_confidence'], df['company_confidence_estimation'], target_names=('0', '1', '2')))

             precision    recall  f1-score   support

          0       0.47      0.86      0.61       100
          1       0.71      0.38      0.50       163
          2       0.78      0.74      0.76       135

avg / total       0.67      0.62      0.61       398



Debugging: Highlight wrong values

In [7]:
df.set_index(['url'], inplace=True)
df = df.dropna(subset =["sentiment"])
def highlight_different(s):
    if s['company_confidence'] != s['company_confidence_estimation']:
        return ['background-color: red' if v == s['company_confidence_estimation'] else '' for v in s ]
    return ['' for v in s]
df2 = df[['company_confidence', 'company_confidence_estimation', 'company_fraction']]
df2.style.apply(highlight_different, axis=1)

,company_confidence,company_confidence_estimation,company_fraction
url,,,
http://bellona.org/news/arctic/2017-04-trump-like-friend-of-putin-tells-him-climate-change-isnt-man-made,0,0,0
http://bigthink.com/robby-berman/even-if-some-people-deny-its-real-corporations-are-stepping-up,1,1,0.0789474
http://bruegel.org/2017/06/nord-stream-2-can-wait/,1,0,0
http://cenblog.org/the-chemical-notebook/2014/03/basfs-climate-change-schizophrenia/,2,2,0.631579
http://climatecasechart.com/case/people-state-california-v-bp-plc-oakland/,1,1,0.0446429
http://enb.iisd.org/climate/cop12/enbots/9nov.html,1,0,0
http://europe.autonews.com/article/20170605/COPY/306059937/automakers-stand-by-paris-accords-goals,1,1,0.0384615
http://fortune.com/2016/05/17/bill-ford-brainstorm-e-future/,2,2,0.434783
http://fortune.com/2017/03/29/exxon-mobil-donald-trump-paris-agreement-climate-change/,2,1,0.0625


## Climate Confidence Baseline:  
Based on BoW
Distribution:

In [8]:
df['climate_confidence'].value_counts(normalize=True)

2.0    0.515267
1.0    0.438931
0.0    0.045802
Name: climate_confidence, dtype: float64

In [9]:
X = df["text"]
y = df["climate_confidence"]
test_size = 0.2
tf_params = {"lowercase": True,
             "analyzer": "char_wb",
             "stop_words": stop,
             "ngram_range": (3, 3),
             "min_df": 1,
             "max_df": 1.0,
             "preprocessor": None,
             "max_features": 3500,
             "norm": None,
             "use_idf": True
             }
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42,
                                                    stratify=y)
priors = y_train.value_counts(normalize=True).values
vectorizer = TfidfVectorizer(**tf_params)
train = vectorizer.fit_transform(X_train)
test = vectorizer.transform(X_test)
clf = MultinomialNB()
clf.fit(train.toarray(), y_train)
pred = clf.predict(test.toarray())
print(classification_report(y_test, pred, target_names=('0', '1', '2')))

             precision    recall  f1-score   support

          0       1.00      0.33      0.50         3
          1       0.42      0.65      0.51        23
          2       0.50      0.30      0.37        27

avg / total       0.49      0.45      0.44        53



## Sentiment Baseline:  
Using the same BoW technique as above  
I exclude texts that are not related to the company or climate change

In [10]:
target_names=('Strongly Negative', 'Negative', 'Neutral' 'Positive', 'Strongly Positive')
cleaned_df = df[(df['company_confidence'] != 0) & (df['climate_confidence'] != 0)]
print("Total: {} texts".format(len(cleaned_df)))
cleaned_df['sentiment'].value_counts(normalize=True)

Total: 244 texts


Positive             0.438525
Negative             0.266393
Neutral              0.147541
Strongly Positive    0.081967
Strongly Negative    0.065574
Name: sentiment, dtype: float64

In [11]:
#Temporary remove Neutral as its too small

X = cleaned_df["text"]
y = cleaned_df["sentiment"]
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
print(f"Num. of train: {len(X_train)}, Num. of test: {len(X_test)}")

Num. of train: 195, Num. of test: 49


In [12]:
tf_params = {"lowercase": True,
             "analyzer": "char_wb",
             "stop_words": stop,
             "ngram_range": (3, 3),
             "min_df": 1,
             "max_df": 1.0,
             "preprocessor": None,
             "max_features": 3500,
             "norm": None,
             "use_idf": True
             }
priors = y_train.value_counts(normalize=True).values
vectorizer = TfidfVectorizer(**tf_params)
train = vectorizer.fit_transform(X_train)
test = vectorizer.transform(X_test)
clf = MultinomialNB()
clf.fit(train.toarray(), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Test prediction

In [13]:
pred = clf.predict(test.toarray())
print(classification_report(y_test, pred, target_names=('Strongly Negative', 'Negative', 'Neutral' 'Positive', 'Strongly Positive')))

                   precision    recall  f1-score   support

Strongly Negative       0.57      0.67      0.62        12
         Negative       0.67      0.40      0.50        10
  NeutralPositive       0.64      0.80      0.71        20
Strongly Positive       0.00      0.00      0.00         3

      avg / total       0.59      0.61      0.59        49



/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 5, does not match size of target_names, 4
  .format(len(labels), len(target_names))
